#### [ 이미지를 파일 데이터셋으로 생성 ]
- 이미지 파일 raw data 사용
- 데이터셋 파일 : csv, json, ... 등등 개발자 선택
- 데이터셋 파일 구성 : 라벨 + 피쳐, 피쳐 + 라벨
- 1개의 행(row)에 1개 이미지가 저장됨 

[1] 모듈로딩 및 데이터 준비 <hr>

In [1]:
## [1-1] 모듈 로딩
import cv2                        ## 일반적인 이미지 파일들에서 순수 이미지 데이터 다루는 모델 
import matplotlib.pyplot as plt   ## 시각화 
import numpy as np                ## 순수 이미지 데이터의 저장 형식 ndarray
import os                         ## 폴더, 파일, 경로 관련 

In [2]:
## [1-2] 데이터 준비
IMG_DIR  = '../data/image/'                 ## 이미지 존재 폴더
ROW, COL = 60, 60                           ## 이미지 리사이징 크기 
DS_DIR   = '../data/image/DS/'              ## 학습에 사용할 이미지 저장 폴더 
IMG_CSV  = '../data/picture_color.csv'      ## 이미지 라벨과 이미지 로우데이터 저장 파일 

DEBUG_MODE = True                           ## 개발 단계에서 코드 체크 용도  

In [3]:
## 학습용 이미지 저장 폴더 생성 
if not os.path.exists(DS_DIR):
    os.mkdir(DS_DIR)
else:
    print(f'{DS_DIR} 존재함')

../data/image/DS/ 존재함


In [4]:
## [1-3] 사용 이미지 파일리스트 추출 
IMG_FILELIST = os.listdir(IMG_DIR)

for filename in IMG_FILELIST:
    if not os.path.isfile(IMG_DIR+filename):
        # 파일이 아닌 경우 처리 리스트에서 제거
        IMG_FILELIST.remove(filename)

if DEBUG_MODE : print(IMG_FILELIST) 

['guellpark.jpg', 'prague.jpg', 'prague_gray.jpg', 'tulip.png', '_resize_guellpark.jpg']


In [5]:
for filename in IMG_FILELIST:
    ## 파일명에서 확장자 추출
    idxValue=filename.find('.')
    if idxValue>0:
        if filename[idxValue+1: ] not in ['jpg', 'jpeg', 'png', 'bmp']:
            IMG_FILELIST.remove(filename) 
    else:
        print(f'NOT FILE : {filename}')
        IMG_FILELIST.remove(filename) 
        
print(IMG_FILELIST)

['guellpark.jpg', 'prague.jpg', 'prague_gray.jpg', 'tulip.png', '_resize_guellpark.jpg']


[2] 이미지 데이터 로딩<hr>

In [6]:
## RGB Color 형식으로 이미지 데이터 로딩 
with open(IMG_CSV, mode='a') as csvF: 

    for filename in IMG_FILELIST:
        # - 3채널로 이미지 로딩 
        rgbIMG = cv2.imread(IMG_DIR+filename, cv2.IMREAD_COLOR)
        print(f' rgbIMG => {rgbIMG.shape}')

        # - 사이즈 조정
        rgbIMG = cv2.resize(rgbIMG, (COL, ROW))
        print(f'rgbIMG => {rgbIMG.shape}')

        # - 파일로 저장 
        resizeFile = f'{DS_DIR}_resize_{filename}'
        print(f'resizeFile => {resizeFile}') 
        
        ## - openCV 채널 순서 BGR ==> 일반 RGB 
        ret = cv2.imwrite(resizeFile, cv2.cvtColor(rgbIMG, cv2.COLOR_BGR2RGB))
        if not ret: print(f'Fail => {resizeFile}') 

        ## - 채널 분리
        b, g, r = cv2.split(rgbIMG)
        b, g, r = b.reshape(-1), g.reshape(-1), r.reshape(-1) 

        # ## - CSV 파일 쓰기 => 라벨1, 픽셀1, 픽셀2, ..., 픽셀n
        label=filename[:5]
        data = np.concatenate((r, g, b))
        print(f'label => {label}, data => {data.shape}')

        sdata = list(map(lambda n: str(n), data))
        print(f'sdata => {sdata[:30]}')

        csvF.write(str(label)+",")          # 숫자 라벨 쓰기
        csvF.write(','.join(sdata) + "\n")  # 리스트 이미지 데이터 -> 문자열 변환 


 rgbIMG => (1800, 2880, 3)
rgbIMG => (60, 60, 3)
resizeFile => ../data/image/DS/_resize_guellpark.jpg
label => guell, data => (10800,)
sdata => ['0', '0', '2', '2', '2', '0', '2', '0', '0', '2', '0', '0', '1', '2', '1', '2', '2', '1', '4', '2', '2', '0', '0', '3', '3', '0', '1', '1', '1', '1']
 rgbIMG => (1142, 1920, 3)
rgbIMG => (60, 60, 3)
resizeFile => ../data/image/DS/_resize_prague.jpg
label => pragu, data => (10800,)
sdata => ['81', '81', '81', '82', '84', '86', '87', '90', '90', '93', '93', '92', '93', '93', '94', '96', '100', '100', '99', '101', '101', '101', '102', '103', '104', '105', '106', '108', '108', '109']
 rgbIMG => (1142, 1920, 3)
rgbIMG => (60, 60, 3)
resizeFile => ../data/image/DS/_resize_prague_gray.jpg
label => pragu, data => (10800,)
sdata => ['42', '42', '43', '43', '45', '47', '47', '51', '52', '55', '57', '57', '59', '61', '63', '63', '65', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '76', '77', '78']
 rgbIMG => (818, 1280, 3)
rgbIMG => (60, 60

In [7]:
import pandas as pd

In [8]:
colorDF = pd.read_csv(IMG_CSV, header=None)
colorDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 10801 entries, 0 to 10800
dtypes: int64(10800), object(1)
memory usage: 844.0+ KB


In [9]:
colorDF.head(1)

0      1      2      3      4      5      6      7      8      9      ...  \
0  guell      0      0      2      2      2      0      2      0      0  ...   

   10791  10792  10793  10794  10795  10796  10797  10798  10799  10800  
0    249      3     93     39      6     53    226    224    164    140  

[1 rows x 10801 columns]

[3] 관심 영역 ROI 지정 및 저장 <hr>

In [11]:
import cv2, numpy as np

In [12]:
img = cv2.imread('../data/image/prague.jpg')

In [ ]:
## [3-1] 이미지 일부 선택
x, y, w, h = cv2.selectROI('img', img, False)
if w and h:
    roi = img[y:y+h, x:x+w]
    cv2.imshow('cropped', roi)
    cv2.moveWindow('cropped',0, 0)
    cv2.imwrite('./cropped.2.jpg', roi)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()